In [1]:
import pandas as pd
import numpy as np
import os
import os.path
# from functools import reduce
from download_data import get_df 
# hooray nice sublime keymaps!
import my_keymap

pd.set_option('max_columns', 40)
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))

<IPython.core.display.Javascript object>

In [2]:
# our big function that runs for a given set of years and outputs the links and nodes .csv files for our visualization
# applying ammendments is slow because it requires parsing the transaction dates and then grouping them
def process(desired_years, apply_amendments, force_data_download=False):
    # load up our data
    pacs = get_df('cm', desired_years, apply_amendments, force_data_download)
    pac_to_pac_transactions = get_df('oth', desired_years, apply_amendments, force_data_download)
    pac_to_cand_transactions =  get_df('pas2', desired_years, apply_amendments, force_data_download)
    candidates = get_df('cn', desired_years, apply_amendments, force_data_download)
    # there's no header to download for this one, guhh
    # has cand_id, a simple way to do the cand_to_pac_linkages,, and meta stuff like total disbursements and receipts
    pac_summary = get_df('webk', desired_years)
    # official links, not monetary directly
    cand_to_pac_linkages = get_df('ccl', desired_years)



    # drop all the negative transactions, for now, todo, figure out what's up with that, they should probably be dealt with differently anyway?
    # print('all transactions:', len(pac_to_pac_transactions))
    print("all transactions: {:,}".format(len(pac_to_pac_transactions)))

    # negative_pac_to_pac_transactions = pac_to_pac_transactions = pac_to_pac_transactions[pac_to_pac_transactions['TRANSACTION_AMT'] < 0]
    pac_to_pac_transactions = pac_to_pac_transactions[pac_to_pac_transactions['TRANSACTION_AMT'] >0]
    # print('only positive transactions:', len(pac_to_pac_transactions))
    print("only positive transactions: {:,}".format(len(pac_to_pac_transactions)))



    # group/index the transactions by source and recipient/target
    pac_to_pac_links  = pac_to_pac_transactions.groupby(['CMTE_ID', 'OTHER_ID']) 
    # and then sum that
    pac_to_pac_links = pac_to_pac_links['TRANSACTION_AMT'].sum()
    # and now reset the index to get rid of the group by and just make it flat
    pac_to_pac_links = pac_to_pac_links.reset_index()


    print("pac_to_pac_transactions: {:,}".format(len(pac_to_pac_transactions)))
    # net transactions are what I'm calling links
    print("pac_to_pac_links: {:,}".format(len(pac_to_pac_links)))
    # print(len(pac_to_pac_links))
    pac_to_pac_links[0:3]


    # ok now on to from pacs to candidates

    # drop the negative for these too
    print("all p2c transactions: {:,}".format(len(pac_to_cand_transactions)))


    pac_to_cand_transactions = pac_to_cand_transactions[pac_to_cand_transactions['TRANSACTION_AMT'] >0]
    print("only positive p2c transactions: {:,}".format(len(pac_to_cand_transactions)))

    # pac_to_cand_transactions[0:3]


    #  this time group by committee and the the candidate. there's an other_id present, not sure what that represents
    pac_to_cand_links  = pac_to_cand_transactions.groupby(['CMTE_ID', 'CAND_ID']) 
    # and then sum that
    pac_to_cand_links = pac_to_cand_links['TRANSACTION_AMT'].sum()
    # and now reset the index to get rid of the group by and just make it flat
    pac_to_cand_links = pac_to_cand_links.reset_index()


    print("pac_to_cand_transactions: {:,}".format(len(pac_to_cand_transactions)))

    print("pac_to_cand_links: {:,}".format(len(pac_to_cand_links)))
    # print(len(pac_to_cand_links))
    # pac_to_cand_links[0:3]

    # adding this, official links
    official_pac_to_cand_linkage = cand_to_pac_linkages[['CMTE_ID', 'CAND_ID']].copy()
    official_pac_to_cand_linkage.columns = ['source', 'target']
    official_pac_to_cand_linkage['sum'] = 0
    official_pac_to_cand_linkage['official'] = True    
    # pac_to_cand_links.sort_values('TRANSACTION_AMT', ascending=False)[0:4]

    # standardize the column names
    pac_to_cand_links.columns = ['source', 'target', 'sum']
    pac_to_pac_links.columns  = ['source', 'target', 'sum']

    # hooray, every link between a pac and another pac, and between a pac and a candidate
    # we'll export this

    # now sort this and make some top 
    every_link = pd.concat([pac_to_cand_links, pac_to_pac_links, official_pac_to_cand_linkage], ignore_index=True).sort_values('sum', ascending=False)

    every_link.reset_index(inplace=True)
    # every_link[0:10]
    top_1k_link  = every_link[0:1000]
    top_10k_link = every_link[0:10000]


    # this is kinda dumb, but it works. 
    # we're rebuilding what we had, but this way it's exhaustive, and the data is so messy, and the force directed graph is so picky it makes sense 
    # actually this isn't dumb, because we need to add fields from the webk

    # OK, now use that to get every possible node, d3-force is very picky
    every_node = pd.DataFrame(np.unique(every_link[['source', 'target']].values))
    every_node.columns = ['id']

    # and same for these, just the unique ones in the top 1k links
    # top_1k_node = pd.DataFrame(np.unique(top_1k_link[['source', 'target']].values))
    # top_1k_node.columns = ['id']

    # # and top 10k
    # top_10k_node = pd.DataFrame(np.unique(top_10k_link[['source', 'target']].values))
    # top_10k_node.columns = ['id']


    # now to more easily add information about the nodes, set up our pacs and candidates with indexes
    candidates.set_index('CAND_ID', inplace=True)
    pacs.set_index('CMTE_ID',inplace=True)
    pac_summary.set_index('CMTE_ID',inplace=True)

    # helpers for the lambdas
    # XXX this is fairly un-pandas, it's be better to join or merge, todo
    def name_from_id(id):
        if (pacs.index.contains(id)):
            return pacs['CMTE_NM'][id]
        elif (candidates.index.contains(id)):
            return candidates['CAND_NAME'][id]
        else:
            return

    def party_from_id(id):
        if (pacs.index.contains(id)):
            return pacs['CMTE_PTY_AFFILIATION'][id]
        elif (candidates.index.contains(id)):
            return candidates['CAND_PTY_AFFILIATION'][id]
        else:
            return

    # todo, using the actual linkage file here would be better, but the cand_id field seems pretty good, refactoring to use merges would fix this
    def official_cand_linkage_from_pac_id(id):
        if (pacs.index.contains(id)):
            return pacs['CAND_ID'][id]
        else:
            return


    def get_total_disb_from_id(id):
        if (pac_summary.index.contains(id)):
            return pac_summary['TTL_DISB'][id]
        else:
            return
        
    def get_total_recs_from_id(id):
        if (pac_summary.index.contains(id)):
            return pac_summary['TTL_RECEIPTS'][id]
        else:
            return

    # set up the name using the pacs and candidates df in the func above
    every_node['name'] = every_node.apply(lambda row: name_from_id(row.id), axis=1) 
    # and party 
    every_node['party'] = every_node.apply(lambda row: party_from_id(row.id), axis=1) 
    # and the bioguide id, for candidate photos
    every_node['bioguide_id'] = every_node.apply(lambda row: get_bioguide_id_from_fec_id(row.id), axis=1) 
    every_node['pac_cand_id'] = every_node.apply(lambda row: official_cand_linkage_from_pac_id(row.id), axis=1) 
    every_node['disb'] = every_node.apply(lambda row: get_total_disb_from_id(row.id), axis=1) 
    every_node['recs'] = every_node.apply(lambda row: get_total_recs_from_id(row.id), axis=1) 
    # if we wanted to do the top10k links and their relevant nodes, this would be the place

    # set up a string for the year for the name of the file
    year_string = ''
    if len(desired_years) is 1:
        year_string = str(desired_years[0])
    else:
        year_string = str(desired_years[0])+'-'+str(desired_years[-1])

    # and finally export it
    every_node.to_csv('out/every_node_'+year_string+'.csv', index=False)
    every_link.to_csv('out/every_link_'+year_string+'.csv', index=False)
    print('done, wrote every_node and every_link .csv files for '+year_string)
    return [every_node, every_link]


# XXX this doesn't directly relate to the FEC data, but it makes it possible to load photos in the vis
# lets try to get the bio id for a given fec candidate id
# helper, simpler than the other, not a zip, no concat
# basically just caches the file locally
def get_df_from_url(file_url):
    file_name = file_url.split('/')[-1]
    # print(file_name)
    if not os.path.isfile(file_name):
        ! curl -L $file_url > $file_name
    df = pd.read_csv(file_name,  sep=',', low_memory=False)
    return df

every_legis = get_df_from_url('https://theunitedstates.io/congress-legislators/legislators-current.csv')
# legis[0:5]
# print(len(every_legis))
hist_url = 'https://theunitedstates.io/congress-legislators/legislators-historical.csv'
hist_legis = get_df_from_url(hist_url)
# print(len(hist_legis))
every_legis = every_legis.append(hist_legis, ignore_index=True)
# print(len(every_legis))

legis = every_legis.dropna(subset=['fec_ids', 'bioguide_id'])

# print(len(legis))

def get_bioguide_id_from_fec_id(fec_id):
    res = legis[legis.fec_ids.str.contains(fec_id)]
#     print(res)
    if len(res) is 1:
        res = res.reset_index()
        return res.bioguide_id[0]
    else:
        return 

In [3]:
print('Begin!')
print ([2018])
# In general, we don't care about what this returns, since it writes files out, but for debugging it's useful 
[nodes, links] = process([2018], apply_amendments=True, force_data_download=False)

Begin!
[2018]
loading: cm18.zip
loading: oth18.zip


/home/jovyan/work/download_data.py:100: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  result_df = result_df.loc[result_df.groupby('TRAN_ID')['TRANSACTION_DT'].idxmax()]


loading: pas218.zip
loading: cn18.zip
loading: webk18.zip
loading: ccl18.zip
all transactions: 565,554
only positive transactions: 546,092
pac_to_pac_transactions: 546,092
pac_to_pac_links: 165,568
all p2c transactions: 203,946
only positive p2c transactions: 197,141
pac_to_cand_transactions: 197,141
pac_to_cand_links: 103,988


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:84: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



done, wrote every_node and every_link .csv files for 2018


In [4]:
# len( nodes.pac_cand_id.astype('str').unique() )
links[0:3]

,index,official,source,sum,target
0,261328,NaN,C00580100,"31,867,499.00",C00618371
1,253843,NaN,C00545947,"29,826,976.00",C00002931
2,139293,NaN,C00075820,"29,473,702.00",C00545947


In [5]:
print([2016])
process([2016], True)

print([2014])
process([2014], True)

print([2012])
process([2012], True)

print([2010])
process([2010], True)

print([2008])
process([2008], True)

[2016]
loading: cm16.zip
downloading from:  https://www.fec.gov/files/bulk-downloads/2016/cm16.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   154  100   154    0     0    690      0 --:--:-- --:--:-- --:--:--   690
100  723k  100  723k    0     0   404k      0  0:00:01  0:00:01 --:--:--  557k
loading: oth16.zip
downloading from:  https://www.fec.gov/files/bulk-downloads/2016/oth16.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   154  100   154    0     0    687      0 --:--:-- --:--:-- --:--:--   684
100 75.7M  100 75.7M    0     0  1324k      0  0:00:58  0:00:58 --:--:-- 1335kk      0  0:00:58  0:00:48  0:00:10 1537k


/home/jovyan/work/download_data.py:100: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  result_df = result_df.loc[result_df.groupby('TRAN_ID')['TRANSACTION_DT'].idxmax()]


loading: pas216.zip
downloading from:  https://www.fec.gov/files/bulk-downloads/2016/pas216.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   154  100   154    0     0    553      0 --:--:-- --:--:-- --:--:--   557
100 14.6M  100 14.6M    0     0  1373k      0  0:00:10  0:00:10 --:--:-- 1360k
loading: cn16.zip
downloading from:  https://www.fec.gov/files/bulk-downloads/2016/cn16.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   154  100   154    0     0    770      0 --:--:-- --:--:-- --:--:--   766773
100  273k  100  273k    0     0   207k      0  0:00:01  0:00:01 --:--:-- 1044k
loading: webk16.zip
downloading from:  https://www.fec.gov/files/bulk-downloads/2016/webk16.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:84: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



done, wrote every_node and every_link .csv files for 2016
[2014]
loading: cm14.zip
downloading from:  https://www.fec.gov/files/bulk-downloads/2014/cm14.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   154  100   154    0     0    678      0 --:--:-- --:--:-- --:--:--   675
100  609k  100  609k    0     0   381k      0  0:00:01  0:00:01 --:--:-- 1199k
loading: oth14.zip
downloading from:  https://www.fec.gov/files/bulk-downloads/2014/oth14.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   154  100   154    0     0    703      0 --:--:-- --:--:-- --:--:--   703
100 18.6M  100 18.6M    0     0  1601k      0  0:00:11  0:00:11 --:--:-- 1945k
loading: pas214.zip
downloading from:  https://www.fec.gov/files/bulk-downloads/2014/pas214.zip
  % Total    % Receiv

pac_to_cand_transactions: 261,780
pac_to_cand_links: 118,573
done, wrote every_node and every_link .csv files for 2010
[2008]
loading: cm08.zip
downloading from:  https://www.fec.gov/files/bulk-downloads/2008/cm08.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   154  100   154    0     0    855      0 --:--:-- --:--:-- --:--:--   855
100  493k  100  493k    0     0   286k      0  0:00:01  0:00:01 --:--:--  489k
loading: oth08.zip
downloading from:  https://www.fec.gov/files/bulk-downloads/2008/oth08.zip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   154  100   154    0     0    785      0 --:--:-- --:--:-- --:--:--   785
100 22.8M  100 22.8M    0     0  1989k      0  0:00:11  0:00:11 --:--:-- 1736k    0  0:00:10  0:00:07  0:00:03 2384k
loading: pas208.zi

[              id                                               name party  \
 0      C00000000                                               None  None   
 1      C00000042  ILLINOIS TOOL WORKS FOR BETTER GOVERNMENT COMM...   NaN   
 2      C00000059                                 HALLMARK CARDS PAC   UNK   
 3      C00000125                                               None  None   
 4      C00000422  AMERICAN MEDICAL ASSOCIATION POLITICAL ACTION ...   NaN   
 5      C00000489              D R I V E POLITICAL FUND, CHAPTER 886   NaN   
 6      C00000547  KANSAS MEDICAL SOCIETY POLITICAL ACTION COMMITTEE   UNK   
 7      C00000638  INDIANA STATE MEDICAL ASSOCIATION POLITICAL AC...   NaN   
 8      C00000729             AMERICAN DENTAL POLITICAL ACTION CMTE.   UNK   
 9      C00000794                               LENT  & SCRIVNER PAC   UNK   
 10     C00000885  INTERNATIONAL UNION OF PAINTERS AND ALLIED TRA...   UNK   
 11     C00000901  BUILD POLITICAL ACTION COMMITTEE OF THE NATIO

In [6]:
# print([2012, 2014, 2016, 2018])
# process([2016, 2018])